<a href="https://colab.research.google.com/github/RosanaR2017/Coursera_Capstone/blob/master/Capstone_Project_Week_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install folium
!pip install PyPDF2
!pip install urllib3
!pip install pdfminer
!pip3 install table_ocr

## 1. Importing Libraries:

In [ ]:
import numpy as np # library to handle data in a vectorized manner
from pprint import pprint

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML documents
from urllib.request import urlopen

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

#Library to extract text and data from pdf format
import PyPDF2
import urllib3
from io import StringIO
import io

print('Libraries imported.')

Libraries imported.


## 2. Description of the problem 

"*Ecuador has one of the highest per capita Covid-19 deaths rates in the world, according to research from Johns Hopkins University. A New York Times analysis of over two dozen countries found that Ecuador had one of the highest excess death rate* "(Noath S,2020) . Therefore, as a current resident of Ecuador, I am concerned about my family safety in this country ,and I want to where are located the highest spot of Covid in this country. This information will help me to travel within this country and know where I can stay safely from this brutal disease. 

**Reference entry:**
url: Noah Smith, [Covid-19 in Ecuador: An Exclusive Q&A with Ecuador’s Ambassador to the U.S.](https://reliefweb.int/report/ecuador/covid-19-ecuador-exclusive-qa-ecuador-s-ambassador-us)


### 3. Description of Data:
I am usig several input of data:

1.   The first input of data that I am using is from wikipedia. Specifically, it is a page with data about COVID-19 pandemic in Ecuador.
2.   The second input of data are the geolocation data of each provinces in Ecuador. This information was extracted using geopy locator.
3. The final data are the nearby venues that were extracted using the API foursquare. 




### Using web scrapping and beautiful soup to extract data from a wikipedia page.

In [ ]:
wiki_URL="https://en.wikipedia.org/wiki/COVID-19_pandemic_in_Ecuador"

In [ ]:
#from inspecting element you get the class wikitable sortable jquery-tablesorter
response=requests.get(wiki_URL)

In [ ]:
soup = BeautifulSoup(response.text,'lxml')
table=soup.find('table',{'class':'wikitable'}).tbody

### First, I cleaned the HTML headers by converting it to text and using remove_html tags function.


In [ ]:
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile(r'(<[^\>]*>)|(\n)|(\xa0)') 
    return re.sub(clean, '', text) 

In [ ]:
header=[]
for th in table.find_all('th'):
  header.append(str(th))
header=list(map(remove_html_tags,header[0:4]))
pprint(header)

['Provinces', 'Confirmed cases', 'Confirmed deaths', 'Probable deaths']


In [ ]:
rows = []
for td in table.find_all('td'):
    rows.append(str(td))
rows=list(map(remove_html_tags,rows))
print(rows, sep=",")

['Azuay', '10,658', '195', '12', 'Bolívar', '2,111', '66', '12', 'Cañar', '2,110', '83', '7', 'Carchi', '3,053', '104', '1', 'Chimborazo', '2,516', '315', '119', 'Cotopaxi', '4,575', '281', '61', 'El Oro', '6,477', '466', '185', 'Esmeraldas', '4,271', '216', '51', 'Galápagos', '227', '1', '1', 'Guayas', '22,263', '1,737', '1,651', 'Imbabura', '4,773', '170', '7', 'Loja', '6,227', '222', '44', 'Los Ríos', '4,017', '342', '237', 'Manabí', '11,284', '1,025', '1,033', 'Morona Santiago', '3,008', '22', '0', 'Napo', '1,440', '74', '2', 'Orellana', '1,956', '53', '17', 'Pastaza', '2,258', '60', '15', 'Pichincha', '59,477', '1,789', '242', 'Santa Elena', '1,738', '368', '274', 'Santo Domingo de los Tsáchilas', '5,293', '361', '116', 'Sucumbíos', '2,763', '92', '2', 'Tungurahua', '5,119', '276', '223', 'Zamora Chinchipe', '1,580', '53', '1']


After cleaning our colums and rows, I am going to construct the dataframe. 

1. First, I construct a list with the provinces  with its data  where each province has it owns data contained in a list. 

In [ ]:
dict_kv=[]
for i in range(0, len(rows)):
  if (rows[i].strip().isalpha()):  
    dict_kv.append([rows[i],rows[i+1],rows[i+2],rows[i+3]])
    
  
dict_kv.append([rows[92],rows[93],rows[94],rows[95]])  # I need to accomodate the last item of the list to get the complete items
      
pprint(dict_kv)
type(dict_kv)

[['Azuay', '10,658', '195', '12'],
 ['Bolívar', '2,111', '66', '12'],
 ['Cañar', '2,110', '83', '7'],
 ['Carchi', '3,053', '104', '1'],
 ['Chimborazo', '2,516', '315', '119'],
 ['Cotopaxi', '4,575', '281', '61'],
 ['Esmeraldas', '4,271', '216', '51'],
 ['Galápagos', '227', '1', '1'],
 ['Guayas', '22,263', '1,737', '1,651'],
 ['Imbabura', '4,773', '170', '7'],
 ['Loja', '6,227', '222', '44'],
 ['Manabí', '11,284', '1,025', '1,033'],
 ['Napo', '1,440', '74', '2'],
 ['Orellana', '1,956', '53', '17'],
 ['Pastaza', '2,258', '60', '15'],
 ['Pichincha', '59,477', '1,789', '242'],
 ['Sucumbíos', '2,763', '92', '2'],
 ['Tungurahua', '5,119', '276', '223'],
 ['Zamora Chinchipe', '1,580', '53', '1']]


list

In [ ]:
df = pd.DataFrame(data=dict_kv, columns = header)
df

,Provinces,Confirmed cases,Confirmed deaths,Probable deaths
0,Azuay,"10,658",195,12
1,Bolívar,"2,111",66,12
2,Cañar,"2,110",83,7
3,Carchi,"3,053",104,1
4,Chimborazo,"2,516",315,119
5,Cotopaxi,"4,575",281,61
6,Esmeraldas,"4,271",216,51
7,Galápagos,227,1,1
8,Guayas,"22,263","1,737","1,651"
9,Imbabura,"4,773",170,7


#### Then, I want to get the coordinates of Ecuador and its provices.

In [ ]:
address = 'Ecuador'

geolocator = Nominatim(user_agent="ecuador_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the country are {}, {}. This country is {}.'.format(latitude, longitude, location))

The geograpical coordinate of the country are -1.3397668, -79.3666965. This country is Ecuador.


Create a map of Ecuador

In [ ]:
# create map of Toronto using latitude and longitude values
map_ecuador = folium.Map(location=[latitude, longitude], tiles='OpenStreetMap', zoom_start=12)
map_ecuador

Then, I want to obtain the latitude and longitude for each provice in ecuador

In [ ]:
address_list=[]
latitude=[]
longitude=[]

for x in df['Provinces']:
  address=str(x)+",Ecuador"
  geolocator = Nominatim(user_agent="ecuador_explorer")
  location = geolocator.geocode(address)
  latitude.append(location.latitude)
  longitude.append(location.longitude)
  
df2= pd.DataFrame({'Latitude':latitude,'Longitude':longitude})
df2.head()


,Latitude,Longitude
0,-3.052445,-79.239514
1,-1.539273,-79.081650
2,-2.574484,-78.980468
3,0.837516,-78.207515
4,-1.926263,-78.729780


Then, I joined the two dataframes and show the results.


In [ ]:
ecuador = df.combine_first(df2)
ecuador.sort_values(['Provinces'], inplace=True)
ecuador

,Confirmed cases,Confirmed deaths,Latitude,Longitude,Probable deaths,Provinces
0,"10,658",195,-3.052445,-79.239514,12,Azuay
1,"2,111",66,-1.539273,-79.081650,12,Bolívar
3,"3,053",104,0.837516,-78.207515,1,Carchi
2,"2,110",83,-2.574484,-78.980468,7,Cañar
4,"2,516",315,-1.926263,-78.729780,119,Chimborazo
5,"4,575",281,-0.680595,-78.437764,61,Cotopaxi
6,"4,271",216,0.734362,-79.385887,51,Esmeraldas
7,227,1,-0.628815,-90.363875,1,Galápagos
8,"22,263","1,737",-2.000000,-80.000000,"1,651",Guayas
9,"4,773",170,0.375362,-78.355681,7,Imbabura


The Foursquare API is going to be used to explore the neighborhoods.

In [ ]:
CLIENT_ID = 'CB245Q52IIDUCQ1VXR2HBN5HANNGGSNTWW2CZ0K2Z4PQWAZT' # your Foursquare ID
CLIENT_SECRET = 'M4O5SZNTHG2JBVJSERHF10H2NC5IX3OMC20IIKHDLA2KHFWB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

I decided to explore Guayaquil, which is the provinces where I live.

In [ ]:
print(ecuador[ecuador['Provinces']=='Guayas'].index.values) # First, I get the index of my province.
print(ecuador.loc[8, 'Provinces'])
guayas_latitude=ecuador.loc[8,'Latitude'] #  guayas latitude value
guayas_longitude=ecuador.loc[8,'Longitude']
print("guayas latitude: {}".format(guayas_latitude)) # guayas longitude value
print("guayas longitude: {}".format(guayas_longitude)) # guayas longitude value

[8]
Guayas
guayas latitude: -2.0
guayas longitude: -80.0


Now, let's get the top 100 venues that are Guayas within a radius of 1000 meters

In [ ]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 100000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    guayas_latitude, 
    guayas_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=CB245Q52IIDUCQ1VXR2HBN5HANNGGSNTWW2CZ0K2Z4PQWAZT&client_secret=M4O5SZNTHG2JBVJSERHF10H2NC5IX3OMC20IIKHDLA2KHFWB&v=20180605&ll=-2.0,-80.0&radius=100000&limit=100'

Send the GET request and examine the results

In [ ]:
ec_results = requests.get(url).json()

From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Then, we get the venues.

In [ ]:
ec_venues = ec_results['response']['groups'][0]['items']
    
ec_nearby_venues = json_normalize(ec_venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
ec_nearby_venues =ec_nearby_venues.loc[:, filtered_columns]

# filter the category for each row
ec_nearby_venues['venue.categories'] = ec_nearby_venues.apply(get_category_type, axis=1)

# clean columns
ec_nearby_venues.columns = [col.split(".")[-1] for col in ec_nearby_venues.columns]
ec_nearby_venues.head()


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Supercines,Movie Theater,-2.054415,-79.874699
1,Wine Bar,Wine Bar,-2.099587,-79.874460
2,Embarcadero 41 Fusión,Peruvian Restaurant,-2.098402,-79.875182
3,Plaza Lagos Town Center,Shopping Mall,-2.098301,-79.875011
4,Parque Samanes,Park,-2.103924,-79.903865


100 venues were returned.

In [ ]:
print('{} venues were returned by Foursquare.'.format(ec_nearby_venues.shape[0]))

100 venues were returned by Foursquare.


Explore provinces in Ecuador

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=100000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
ecuador_venues = getNearbyVenues(names=ecuador['Provinces'],
                                   latitudes=ecuador['Latitude'],
                                   longitudes=ecuador['Longitude']
                                  )

Azuay
Bolívar
Carchi
Cañar
Chimborazo
Cotopaxi
Esmeraldas
Galápagos
Guayas
Imbabura
Loja
Manabí
Napo
Orellana
Pastaza
Pichincha
Sucumbíos
Tungurahua
Zamora Chinchipe


Let's check the size of the resulting dataframe

In [ ]:
print(ecuador_venues.shape)

(1236, 7)


In [ ]:
# add markers to map
for lat, lng, provinces, confirmedCases, ConfirmedDeaths in zip(ecuador['Latitude'], ecuador['Longitude'], ecuador['Provinces'], ecuador['Confirmed cases'], ecuador['Confirmed deaths']):
    label = 'Provinces: {}, Confirmed Cases: {}, Deaths: {}'.format(provinces, confirmedCases ,ConfirmedDeaths)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=(5*float(confirmedCases.replace(',',''))/59477),
        popup=label,
        color='red',
        fill='red',
        fill_color='#FF0000',
        fill_opacity=(float(confirmedCases.replace(',',''))/59477),
        parse_html=False).add_to(map_ecuador)
  

    


In [ ]:
map_ecuador

**Conclusion:** With this information, I conclude that Pichincha is most dangerous provinces to visit. 

I can make more further analysis about this data. This analysis will continue on week 5.

In [ ]:
print(ecuador_venues.shape)
ecuador_venues

(1236, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Azuay,-3.052445,-79.239514,Piedra de Agua,-2.922200,-79.063565,Spa
1,Azuay,-3.052445,-79.239514,NOVAQUA - SPA TERMAL,-2.922495,-79.062136,Spa
2,Azuay,-3.052445,-79.239514,Centro Historico,-2.896612,-79.007953,Historic Site
3,Azuay,-3.052445,-79.239514,Parque Nacional El Cajas,-2.890664,-79.054500,Field
4,Azuay,-3.052445,-79.239514,Mangiare Bene,-2.895692,-79.009809,Italian Restaurant
5,Azuay,-3.052445,-79.239514,Hacienda / Hostería Dos Chorreras,-2.783977,-79.168196,Resort
6,Azuay,-3.052445,-79.239514,El Parque Nacional Cajas,-2.816601,-79.266647,National Park
7,Azuay,-3.052445,-79.239514,El Charrua,-2.911503,-79.006685,BBQ Joint
8,Azuay,-3.052445,-79.239514,Parque Calderón,-2.897566,-79.004757,Park
9,Azuay,-3.052445,-79.239514,Noé sushi bar,-2.907668,-79.009867,Sushi Restaurant
